In [1]:
##PACKAGE IMPORTING
import pandas as pd
from dask.distributed import Client, progress ##For Progress Dashboard
import dask.dataframe as dd
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from ggplot import *
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

C:\ProgramData\Anacondanew\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anacondanew\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

In [15]:
#SAVE YOUR MELTED DATAFRAME WITH CHUNCK OPTION
##ARGUMENT CHUNK SIZE IS DEFINED
%time sales_01 = pd.read_csv(r"C:\Users\Selvamani\Desktop\sales_train_validation.csv",chunksize=1000)
def Melt_Save(df,Melt_Vars_List):
    for chunk in tqdm.tqdm_notebook(sales_01):
        result = chunk.melt(id_vars= Melt_Vars_List)
        # export the results into a new file and append for each iteration using mode
        result.to_csv("C:/Users/Selvamani/Desktop/Master/main.csv", index=False , mode='a',header = False)
Melt_Save(sales_01,Melt_Vars_List)

Wall time: 4.98 ms


In [43]:
##PARALLEL PROCESSING STARTS HERE USING DASK
Column_Name_List = ['id','item_id','dept_id','cat_id','store_id','state_id' ,'variable' ,'value']
%time Raw_M_Data = dd.read_csv("C:/Users/Selvamani/Desktop/Master/main.csv",names=Column_Name_List,header=None)
%time print(Raw_M_Data.head())

Wall time: 186 ms
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id variable  value  
0       CA      d_1      0  
1       CA      d_1      0  
2       CA      d_1      0  
3       CA      d_1      0  
4       CA      d_1      0  
Wall time: 5 s


In [59]:
"Finding Top 10 Products Sold On an average per day"
%time Mean_Values = Raw_M_Data.groupby(['store_id']).value.mean().compute()

Wall time: 9min 22s


In [117]:
##WALMART LAST WEEK
df_sales_train=pd.read_csv(r"C:\Users\Selvamani\Desktop\sales_train_validation.csv")
cal=pd.read_csv(r"C:\Users\Selvamani\Desktop\Master\calendar.csv")
cal_train=pd.read_csv(r"C:\Users\Selvamani\Desktop\Master\cal_train.csv")
cal_pred = pd.read_csv(r"C:\Users\Selvamani\Desktop\Master\cal_pred.csv")
print(cal_train.columns)
cal_train=cal_train.drop(['Unnamed: 0', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year','d'],axis=1)
cal_pred=cal_pred.drop(['Unnamed: 0', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year','d'],axis=1)
print(type(cal_train))
cal_d = cal['d'].tolist()
cal_date = cal['date'].tolist()
cal_dict = dict(zip(cal_d, cal_date))
df1_sales_train=df_sales_train.iloc[:,:6]
df2_sales_train = df_sales_train.iloc[:,6:].rename(columns=cal_dict)
df_master=pd.concat([df1_sales_train,df2_sales_train],axis=1).drop('id',axis=1)
df_master.head()

Index(['Unnamed: 0', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'd', 'snap_CA', 'snap_TX', 'snap_WI', 'events_name_1_1',
       'events_name_2_1', 'events_type_1_1', 'events_type_2_1'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>


,item_id,dept_id,cat_id,store_id,state_id,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [118]:
items=df_master['item_id'].unique()
state=df_master['state_id'].unique()
stores=df_master['store_id'].unique()
stores1={'CA_1':1,'CA_2':2,'CA_3':3,'CA_4':4,'TX_1':5,'TX_2':6,'TX_3':7,'WI_1':8,'WI_2':9,'WI_3':10}
state1={'CA':1,'TX':2,'WI':3}
fun=lambda x: 1 if (x == 5) | (x == 6) else (0)
fun1=lambda x:1 if x==True else(0)
df_comp=pd.DataFrame()
result=[]
j=1
start=time.time()
print(start)

1590428037.6249914


In [139]:
j=1
for i in items:
     if j<=1:
        print('entering for')
        print(i)
        df1=df_master.loc[df_master['item_id']== i ]
        df1=pd.melt(id_vars=['store_id'],value_vars=df_master.iloc[:,5:],var_name='Date',value_name='Sales',frame=df1)
        df1=df1.sort_values(['store_id','Date'])
        df1=pd.merge(df1, cal_train, how='left', left_on=['Date'], right_on=['date'])
        df1['dayofyear'] = pd.DatetimeIndex(df1['Date']).dayofyear
        df1['dayofweek'] = pd.DatetimeIndex(df1['Date']).dayofweek
        df1['weekofyear'] = pd.DatetimeIndex(df1['Date']).weekofyear
        df1['weekend'] = df1['dayofweek'].apply(fun)
        df1['monthstart'] = pd.DatetimeIndex(df1['Date']).is_month_start
        df1['monthstart'] = df1['monthstart'].apply(fun1)
        df1['quarterstart'] = pd.DatetimeIndex(df1['Date']).is_quarter_start
        df1['quarterstart'] = df1['quarterstart'].apply(fun1)
        df1['yearstart'] = pd.DatetimeIndex(df1['Date']).is_year_start
        df1['yearstart'] = df1['yearstart'].apply(fun1)
        df1['store_id']=df1['store_id'].map(stores1)
        df1 = df1.drop(['Date','date'], axis=1)
        df1_Sales = df1['Sales']
        df1 = df1.drop('Sales', axis=1)
        df1 = pd.concat([df1, df1_Sales], axis=1)
        predict_df = pd.DataFrame()
        K = 56
        elements = [1,2,3,4,5,6,7,8,9,10]
        predict_df['store_id'] = [ele for ele in elements for i in range(K)]
        predict_df['predict_dates'] = pd.concat([pd.DataFrame(pd.date_range(start='2016/04/25',end='2016/06/19'))]*10,axis=0,ignore_index=True)
        predict_df['dayofyear'] = pd.DatetimeIndex(predict_df['predict_dates']).dayofyear
        predict_df['dayofweek'] = pd.DatetimeIndex(predict_df['predict_dates']).dayofweek
        predict_df['weekofyear'] = pd.DatetimeIndex(predict_df['predict_dates']).weekofyear
        predict_df['weekend'] = predict_df['dayofweek'].apply(fun)
        predict_df['monthstart'] = pd.DatetimeIndex(predict_df['predict_dates']).is_month_start
        predict_df['monthstart'] = predict_df['monthstart'].apply(fun1)
        predict_df['quarterstart'] = pd.DatetimeIndex(predict_df['predict_dates']).is_quarter_start
        predict_df['quarterstart'] = predict_df['quarterstart'].apply(fun1)
        predict_df['yearstart'] = pd.DatetimeIndex(predict_df['predict_dates']).is_year_start
        predict_df['yearstart'] = predict_df['yearstart'].apply(fun1)
        cal_pred['date'] = pd.to_datetime(cal_pred['date'])
        Final_Prediction_File = pd.merge(cal_pred,predict_df, how='left', left_on=['date'], right_on=['predict_dates'])
        Final_Prediction_File = Final_Prediction_File.drop(['predict_dates','date'],axis=1)
        Final_Prediction_File = Final_Prediction_File.reindex(columns=['store_id', 'snap_CA', 'snap_TX', 'snap_WI', 'events_name_1_1',
       'events_name_2_1', 'events_type_1_1', 'events_type_2_1', 'dayofyear',
       'dayofweek', 'weekofyear', 'weekend', 'monthstart', 'quarterstart',
       'yearstart'])
        print(Final_Prediction_File.head())
        print(Final_Prediction_File.columns)
        print(df1.head())
        print(df1.columns)
        X=df1.drop('Sales',axis=1)
        y=df1['Sales']
        print("ENTERING MODEL")
        xgb=XGBRegressor()
        xgb.fit(X,y)
        y_pred=xgb.predict(Final_Prediction_File)
       # df_comp['y_test']=y_test
       # df_comp['y_pred']=y_pred
        #df_comp['y_pred']=df_comp['y_pred'].round()
       # rmse=np.sqrt(mean_squared_error(y_test,np.round(y_pred)))
       # result.append(rmse)
        print(y_pred)
        j=j+1        


entering for
HOBBIES_1_001
   store_id  snap_CA  snap_TX  snap_WI  events_name_1_1  events_name_2_1  \
0         1        0        0        0                0                0   
1         2        0        0        0                0                0   
2         3        0        0        0                0                0   
3         4        0        0        0                0                0   
4         5        0        0        0                0                0   

   events_type_1_1  events_type_2_1  dayofyear  dayofweek  weekofyear  \
0                0                0        116          0          17   
1                0                0        116          0          17   
2                0                0        116          0          17   
3                0                0        116          0          17   
4                0                0        116          0          17   

   weekend  monthstart  quarterstart  yearstart  
0        0           0     

C:\ProgramData\Anacondanew\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anacondanew\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[23:47:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[ 0.22542691  0.22575551  0.29786754  0.2635852   0.12427363  0.15884635
  0.15884635  0.05970696  0.05218282  0.08205882  0.22542691  0.22575551
  0.29786754  0.2635852   0.12427363  0.15884635  0.15884635  0.05970696
  0.05218282  0.08205882  0.21669713  0.21702573  0.28913775  0.25485545
  0.1237995   0.15837222  0.15837222  0.05923283  0.0517087   0.08158469
  0.19942027  0.19974887  0.2718609   0.24247018  0.1229409   0.15751362
  0.15751362  0.05837423  0.05085009  0.08072609  0.20002496  0.2087059
  0.28807366  0.24269944  0.13105345  0.16562617  0.16562617  0.08060643
  0.07308233  0.10295826  0.26911327  0.2777942   0.33844233  0.278472
  0.15542307  0.1899958   0.1899958   0.10464558  0.09712145  0.12699741
  0.24767098  0.2563519   0.33695582  0.2716257   0.14857677  0.18314949
  0.18314949  0.09779927  0.09027514

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 559
Data columns (total 16 columns):
store_id           560 non-null int64
predict_dates      560 non-null datetime64[ns]
dayofyear          560 non-null int64
dayofweek          560 non-null int64
weekofyear         560 non-null int64
weekend            560 non-null int64
monthstart         560 non-null int64
quarterstart       560 non-null int64
yearstart          560 non-null int64
snap_CA            560 non-null int64
snap_TX            560 non-null int64
snap_WI            560 non-null int64
events_name_1_1    560 non-null int64
events_name_2_1    560 non-null int64
events_type_1_1    560 non-null int64
events_type_2_1    560 non-null int64
dtypes: datetime64[ns](1), int64(15)
memory usage: 74.4 KB
